## Exercise 7

Build your own CNN and try to achieve the highest possible accuracy on MNIST.

## Approach

As per [Caffe2's MNIST tutorial](https://caffe2.ai/docs/tutorial-MNIST.html), we are going to build a small CNN based on the LeNet architecture:

Layer  | Type            | Maps | Size    | Kernel size | Stride | Activation
-------|-----------------|------|---------|-------------|--------|-----------
logits | Fully Connected | -    | 10      | -           | -      | softmax
F5     | Fully Connected | -    | 500     | -           | -      | relu
S4     | Max Pooling     | 100  | 4 x 4   | 2 x 2       | 2      | -
C3     | Convolution     | 100  | 8 x 8   | 5 x 5       | 1      | tanh
S2     | Max Pooling     | 20   | 12 x 12 | 2 x 2       | 2      | -
C1     | Convolution     | 20   | 28 x 28 | 5 x 5       | 1      | tanh
X      | Input           | 1    | 28 x 28 | -           | -      | -

In [1]:
import tensorflow as tf

First define the data we will be feeding the network at each training step:

- `X` is a batch of 28 x 28 grayscale (= 1 channel) images. The MNIST dataset contains them as a sequence of 768 pixel values, so we'll first reshape them to a 2D image.
- `y` is a batch of [one-hot encoded](https://en.wikipedia.org/wiki/One-hot) digit classes.

The one-hot encoding simply means that each digit class, which lies in the range 0-9, is represented by a sequence of ten values, all zero except for the index corresponding to the digit class itself.

For instance, the digit class 3's one-hot encoding is `0001000000`, or:

`i`    | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |
-------|---|---|---|---|---|---|---|---|---|---|
`y[i]` | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0

In [2]:
X = tf.placeholder(tf.float32, shape=[None, 28 * 28 * 1])
y = tf.placeholder(tf.int32,   shape=[None, 10])

Then we build the model according to the table above. This should be fairly straightforward, except perhaps for the last layer.

The last layer of the network contains ten output neurons. Like every neuron in the network, they multiply their inputs by their weights and add a bias term:

\begin{equation}
y = \sum{Wx} + b
\end{equation}

But note that they have no activation function. This means that they pass on the result of this calculation to their outputs as-is, and do not first perform a _tanh_ or _relu_ conversion. These output values are therefore not limited to a fixed range, such as [0, 1], but can take on any real number, both positive and negative.

Now, we want to teach each output neuron to output a higher value when it thinks the input image contains "its" digit, and a lower value when it does not.

## Logits and probabilities

We therefore treat these output values as [logits](https://stats.stackexchange.com/questions/52825/what-does-the-logit-value-actually-mean#52836), or _logarithmic odds_. Given a probability $p$, its odds is defined as:

\begin{equation}
\frac{p}{1 - p}
\end{equation}

And a logarithmic odd is simply the natural logarithm of this:

\begin{equation}
L = \ln \frac{p}{1 - p}
\end{equation}

And the other way around:

\begin{equation}
p = \frac{1}{1 + e^{-L}}
\end{equation}

The relationship between a probability $p$ and the logit $L$ can be shown graphically as well:

![Curve](https://i.stack.imgur.com/h6N7o.png)

The thing to take away is we want to teach an output neuron to output a higher _logit_ when it thinks the input image contains "its" digit, since a higher logit corresponds to a higher _probability_.

In [3]:
with tf.name_scope('cnn'):

    # Image: [768] --> [28 x 28 x 1].
    X_reshaped = tf.reshape(X, shape=[-1, 28, 28, 1])
    
    # Image: [28 x 28 x 1] --> [24 x 24 x 20].
    c1 = tf.layers.conv2d(
        X_reshaped,
        filters=20,
        kernel_size=[5, 5],
        strides=[1, 1],
        padding='valid',
        activation=tf.nn.tanh)

    # Image: [24 x 24 x 20] --> [12 x 12 x 20].
    s2 = tf.layers.max_pooling2d(
        c1,
        pool_size=[2, 2],
        strides=[2, 2],
        padding='valid')

    # Image: [12 x 12 x 20] --> [8 x 8 x 100].
    c3 = tf.layers.conv2d(
        s2,
        filters=100,
        kernel_size=[5, 5],
        strides=[1, 1],
        padding='valid',
        activation=tf.nn.relu)

    # Image: [8 x 8 x 100] --> [4 x 4 x 100].
    s4 = tf.layers.max_pooling2d(
        c3,
        pool_size=[2, 2],
        strides=[2, 2],
        padding='valid')

    # Image: [4 x 4 x 100] --> [1600].
    s4_reshaped = tf.reshape(s4, [-1, 4 * 4 * 100])

    # Image: [1600] --> [500].
    f5 = tf.layers.dense(
        s4_reshaped,
        units=500,
        activation=tf.nn.relu)

    # Image: [500] --> [10]
    logits = tf.layers.dense(
        f5,
        units=10,
        activation=None)

In order to learn something, we must first define a loss or cost function. For each image we feed the network, the loss tells us in essence how close to the right answer we are, or how far away.

The larger the difference between the answer given by the network and the desired answer, that is, the ground truth, the higher the loss value. While learning we aim to minimize the loss to as close to zero as possible.

Recall that when we feed the network a single image, we end up with ten _logits_ at the output end of the network. We also have a one-hot encoded sequence of ten values that indicate the desired output. How do we calculate a loss given this combination?

## Softmax

We first apply the _softmax_ function to the ten outputs. This function takes the ten outputs, and normalizes each output value to the _probability_ that the input image belongs to "its" class. The formula is quite simple:

\begin{equation}
p_i = \frac{e^{L_i}}{\sum_{j=0}^9 e^{L_j}}
\end{equation}

This means that for an output neuron at index $i$, it divides the exponential of its (logit) output $L_i$ by the sum of the exponentials of the (logit) outputs of all output neurons.

This action normalizes each output value to a probability in the range [0, 1], and has the property that the sum of all probabilities is 1.

So far, so good. We now have converted the logit values of the ten output neurons to ten probabilities. We now need to define a loss function that will output a low value if the network guesses correctly on an image, that is, when the network assignes a high probability to the right digit class, and a low probability to all others.

## Cross entropy

The _cross entropy_ formula is typically used in this case. The cross entropy function looks like this, for a single image:

\begin{equation}
J = - \sum_{i=0}^9 y_i \log p_i
\end{equation}

Here $y_i$ is a value in the [one-hot encoded](https://en.wikipedia.org/wiki/One-hot) label `y`, meaning it's 1 if the image's digit class is $i$, and 0 everywhere else. The $p_i$ probability is what we just calculated earlier using _softmax_ and is in the range [0, 1].

Suppose the network guessed the digit 3 correctly by assigning probability $p_3 = 1$ and the rest all zeroes. Its contribution to the loss should be zero, and it is:

\begin{equation}
y_3 \times \log p_3 = 1 \times \log 1 = 1 \times 0 = 0
\end{equation}

All the other probabilities ($i \neq 3$), being all zeroes as we desire, should also contribute zero:

\begin{equation}
y_i \times \log p_i = 0 \times \log 0 = 0 \times -\infty = 0
\end{equation}

The total cross entropy $J$ for this one image would therefore be $-0$ or simply zero.

Now, if the network answers incorrectly on this image, by assigning the probability 1 to the digit class 9 and zeroes to all other classes, the contribution of its answer to to the loss should be significant:

\begin{equation}
y_3 \times \log p_3 = 1 \times \log 0 = 1 \times -\infty = -\infty
\end{equation}

And for the wrong answer:

\begin{equation}
y_9 \times \log p_9 = 0 \times \log 1 = 0 \times 0 = 0
\end{equation}

The total cross entropy $J$ for this one image would therefore be $-(-\infty)$ or $+\infty$.

Our loss function is now simply the mean of the losses for all the images we train with, typically in a batch. We'll then aim to minimize this mean to get better and learn.

In [4]:
with tf.name_scope('loss'):

    xentropy = tf.losses.softmax_cross_entropy(
        onehot_labels=y,
        logits=logits)

    loss = tf.reduce_mean(xentropy)

Here's where we'll use a so-called Adam optimizer to minimize the loss function we defined earlier:

In [5]:
learning_rate = 0.001

with tf.name_scope('train'):
    
    optimizer   = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

Now suppose we are training and learning and thereby reduce the loss on the training set. Do we also have an independent measure for how we are on _any_ data? For instance, we'd like to know how good we are on a dedicated test set. We use _accuracy_ to determine this.

## Accuracy

If we feed an image that represents the digit 3, for instance, then we are accurate if output neuron #3 outputs the highest value of all output neurons. Remember that we treat these as _logits_, so a highest output corresponds to the highest probability.

We will use the `in_top_k(k=1)` function of Tensorflow, which will output (per image) a boolean that indicates whether the prediciton for the target class 3 is in the top `k=1` predictions that the network makes. In other words, we wanted the network to predict target class 3, and did its top prediction indicate that class?

Since we are feeding batches of images at a time, this function actually returns a 1D tensor, with booleans for all images in the batch. We cast these booleans to floats (`True` becoming 1.0, and `False` becoming 0.0) and calculate the mean. The higher the mean, the more `True` values we had and the more accurate we are.

Note that since `in_top_k` does not expect a one-hot encoded ground truth, but a simple integer class number, we first convert the `y` by means of the $\arg\max$ function, which returns the index $i$ where $y_i$ has its maximum value (of 1.0 in our case).

In [6]:
with tf.name_scope('eval'):
    
    labels   = tf.argmax(y, axis=1)
    correct  = tf.nn.in_top_k(logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

X_test = mnist.test.images
y_test = mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
from datetime import datetime

now         = datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir = 'tf_logs'
logdir      = '{}/run-{}/'.format(root_logdir, now)

In [9]:
loss_summary      = tf.summary.scalar('loss', loss)
acc_train_summary = tf.summary.scalar('acc_train', accuracy)
acc_test_summary  = tf.summary.scalar('acc_test',  accuracy)

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [10]:
n_epochs   = 5
batch_size = 100
n_batches  = mnist.train.num_examples // batch_size

print("{}: Training...".format(datetime.now()))

with tf.Session() as sess:
    tf.global_variables_initializer().run()

    for epoch in range(n_epochs):
        print("\r{} of {} epochs".format(epoch + 1, n_epochs), end='')

        # Each epoch we train all batches.
        for batch in range(n_batches):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

            # Evaluate the network now and then, so we can visualize
            # its progress in Tensorboard.
            if batch % 10 == 0:
                step = epoch * n_batches + batch
                file_writer.add_summary(
                    loss_summary.eval(feed_dict={X: X_batch, y: y_batch}),
                    step)
                file_writer.add_summary(
                    acc_train_summary.eval(feed_dict={X: X_batch, y: y_batch}),
                    step)
                file_writer.add_summary(
                    acc_test_summary.eval(feed_dict={X: X_test,  y: y_test}),
                    step)
        
print("\r\n{}: All done.".format(datetime.now()))

2017-10-15 13:31:43.416863: Training...
5 of 5 epochs2017-10-15 13:55:28.686547: All done.
